In [118]:
import pandas as pd
from helper import findKRelevant_simple,my_evaluation,alertFinishJob,alertError
import time
import argparse
import pickle
#import sys
from multiprocessing import Pool
import numpy as np
import sklearn.preprocessing as pp
from scipy.sparse import csr_matrix

In [119]:
K = 100
#proc = 8
MAX_tid = 225

In [120]:
# print("Load Playlist-Song matrix")    
df_ps_train = pd.read_hdf('data/df_data/df_playlistSong/df_ps_train_new.hdf')
# df_ps_test = pd.read_hdf('data/df_data/df_playlistSong/df_ps_test_new.hdf')
# df_ps_test_truth = pd.read_hdf('data/df_data/df_playlistSong/df_ps_test_truth_new.hdf')

In [121]:
df_ps_test = pd.read_hdf('data/df_data/df_playlistSong/df_ps_test_new.hdf')

In [122]:
# pid_list_pred = list(df_ps_test.index) 
# pid_test_length = len(pid_list_pred)
# dict_index = {k:v for k,v in zip(range(0,pid_test_length),pid_list_pred)}


In [123]:
sim_path = 'data/cosineSimMatrix_playlist.pickle'
with open(sim_path,'rb') as f:
    sim_matrix = pickle.load(f)    
sim_matrix = sim_matrix.tocsr()   


In [124]:
def inference_row(list_tid, ps_matrix):
#     Normalization not necessary in this case, but may be needed in future
    ps_matrix_norm = pp.normalize(ps_matrix, axis=1)
    length_tid = len(list_tid)
    sparse_row = csr_matrix((np.ones(length_tid), (np.zeros(length_tid), list_tid)), shape=(1, 57884))
    sparse_row = pp.normalize(sparse_row, axis=1)
    
    return sparse_row * ps_matrix_norm.T


def my_function(data):
    
    index_pid = data[0]
    pid = dict_index.get(index_pid)
    current_list = data[1]
    
    start = time.time()
#     print("Pid: ",pid)

#    topK_pid = findKRelevant_simple(pid,df_ps_train,K)
    sim_vector = sim_matrix.getrow(index_pid).toarray()[0].tolist()
    
    # Enumerate index and rating
    counter_list = list(enumerate(sim_vector, 0))

    # Sort by rating
    sortedList = sorted(counter_list, key=lambda x:x[1],reverse=True)
    
    topK_pid = [i for i,_ in sortedList[1:K+1]]
    
    n = 0
    
    while(1):
        
        top_pid = topK_pid[n]
        
#        top_pid = dict_index.get(index_top_pid)
        
        add_tid_list = df_ps_train.loc[top_pid].tid
                
        # Form new list
        new_tid_list = current_list + add_tid_list
            
        # Check number of songs and Add to data for prediction
        total_song = len(new_tid_list)
#            print("n: {}\t total_song: {}".format(n,total_song))
        if (total_song > MAX_tid):
            new_tid_list = new_tid_list[:MAX_tid]            
            # Add
            current_list = new_tid_list
            break
        else:
            current_list = new_tid_list
#        print(n)
        n += 1
#         print(n)
        if (n == K):
            break
        
#    SIZE = SIZE - 1
#     print("Time taken = {0:.5f}".format(time.time() - start))
    
    return [pid,current_list]


def get_best_tid(current_list, ps_matrix_row):
    
    start = time.time()
#     print("Pid: ",pid)

#    topK_pid = findKRelevant_simple(pid,df_ps_train,K)
    sim_vector = inference_row(current_list, ps_matrix_row).toarray()[0].tolist()
    
    # Enumerate index and rating
    counter_list = list(enumerate(sim_vector, 0))

    # Sort by rating
    sortedList = sorted(counter_list, key=lambda x:x[1],reverse=True)
    
    topK_pid = [i for i,_ in sortedList[1:K+1]]
    
    n = 0
    
    while(1):
        
        top_pid = topK_pid[n]
        
#        top_pid = dict_index.get(index_top_pid)
        
        add_tid_list = df_ps_train.loc[top_pid].tid
                
        # Form new list
        new_tid_list = current_list + add_tid_list
            
        # Check number of songs and Add to data for prediction
        total_song = len(new_tid_list)
#            print("n: {}\t total_song: {}".format(n,total_song))
        if (total_song > MAX_tid):
            new_tid_list = new_tid_list[:MAX_tid]            
            # Add
            current_list = new_tid_list
            break
        else:
            current_list = new_tid_list
#        print(n)
        n += 1
#         print(n)
        if (n == K):
            break
        
#    SIZE = SIZE - 1
    print("Time taken = {0:.5f}".format(time.time() - start))
    
    return [current_list]

In [125]:
current_list = list(df_ps_test.loc[pid_list_pred].tid)

In [126]:
# print(len(current_list[1]))

In [127]:
proc = 16

In [128]:
with Pool(proc) as p:
    new_list = p.map(my_function, zip(range(0,pid_test_length),current_list))

In [129]:
pickle_path = 'data/giantMatrix_new.pickle'
with open(pickle_path,'rb') as f:
    ps_matrix = pickle.load(f)
    
ps_matrix_row = ps_matrix.tocsr()

/tmp/ipykernel_5128/1510394896.py:3: DeprecationWarning: Please use `dok_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.dok` namespace is deprecated.
  ps_matrix = pickle.load(f)


In [130]:
df_ps_test_truth = pd.read_hdf("data/df_data/"+"df_playlistSong/df_ps_test_truth_new.hdf")

pid_list_test = list(df_ps_test_truth.index) 


In [131]:
test_track10 = df_ps_test_truth.iloc[0,0]

In [132]:
resultado_final = inference_row(test_track10, ps_matrix_row)

In [133]:
r1 = set(resultado_final.nonzero()[1])

In [134]:
r2 = set(sim_matrix.getrow(0).nonzero()[1])

In [135]:
# print(resultado_final-sim_matrix.getrow(0))

In [136]:
# len(r1.intersection(r2))

with open('dict_uri2tid.pkl', 'rb') as f:
    dict_uri2tid = pickle.load(f)


In [137]:
dict_uri2tid['spotify:track:0UaMYEvWZi0ZqiDOoHU3YI']

4292

In [138]:
# test_track10[0] = 51515115

In [139]:
test_random = [5, 15, 20, 25, 30, 35, 40, 45]

In [144]:
r_f = get_best_tid(test_track10, ps_matrix.tocsr())

Time taken = 0.00336


In [145]:
a = set(new_list[0][1])

In [146]:
b = set(r_f[0])

In [147]:
# It is not all of them since the similarity matrix is built on training set and not the whole set
len(b.intersection(a))

169

In [42]:
# a

In [503]:
# sim_matrix.getrow(0)

In [502]:
# sim_matrix.getrow(10).toarray()[0].tolist()